In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import os
import tensorflow as tf
from tensorflow import keras
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix


LOAD DATA


In [ ]:
train_data_dir = "/content/drive/MyDrive/Đồ Án/DataSet/dog-20240603T032219Z-001/dog/10dogtrain"
test_data_dir = "/content/drive/MyDrive/Đồ Án/DataSet/dog-20240603T032219Z-001/dog/10dogtest"
# Define the dog categories
categories = ["Afghan_hound", "Blenheim_spaniel", "Chihuahua", "Japanese_spaniel", "Maltese_dog",
              "papillon", "Pekinese", "Rhodesian_ridgeback", "Shih-Tzu", "toy_terrier"]


train_category_sizes = {category: 0 for category in categories}
test_category_sizes = {category: 0 for category in categories}

# Create empty lists for training data and labels
x_train = []
y_train = []
# Iterate through each dog category
for category in categories:
    # Path to each dog category
    category_path = os.path.join(train_data_dir, category)
    # Iterate through each image in that category
    for img in os.listdir(category_path):
        try:
            img_path = os.path.join(category_path, img)
            # Read and resize the image
            img = Image.open(img_path)
            img_resized = img.resize((224, 224))  # Resize to 224x224
            img_array = np.array(img_resized)
            # Add the image and label to the lists
            x_train.append(img_array)
            y_train.append(categories.index(category))
            train_category_sizes[category] += 1
        except Exception as e:
            print(f"Error loading image {img_path}: {e}")
# Convert lists to NumPy arrays
x_train = np.array(x_train)
y_train = np.array(y_train)

# Repeat the process for test data
x_test = []
y_test = []
for category in categories:
    category_path = os.path.join(test_data_dir, category)
    for img in os.listdir(category_path):
        try:
            img_path = os.path.join(category_path, img)
            img = Image.open(img_path)
            img_resized = img.resize((224, 224))  # Resize to 224x224
            img_array = np.array(img_resized)
            x_test.append(img_array)
            y_test.append(categories.index(category))
            test_category_sizes[category] += 1
        except Exception as e:
            print(f"Error loading image {img_path}: {e}")
x_test = np.array(x_test)
y_test = np.array(y_test)

print(f"Shape of x_train: {x_train.shape}")
print(f"Shape of x_test: {x_test.shape}")
print(f"Shape of y_train: {y_train.shape}")
print(f"Shape of y_test: {y_test.shape}")

print("Training data category sizes:")
for category, size in train_category_sizes.items():
    print(f"{category}: {size}")
print("Test data category sizes:")
for category, size in test_category_sizes.items():
    print(f"{category}: {size}")

Shape of x_train: (3251, 224, 224, 3)
Shape of x_test: (1775, 224, 224, 3)
Shape of y_train: (3251,)
Shape of y_test: (1775,)
Training data category sizes:
Afghan_hound: 298
Blenheim_spaniel: 397
Chihuahua: 387
Japanese_spaniel: 300
Maltese_dog: 381
papillon: 326
Pekinese: 257
Rhodesian_ridgeback: 306
Shih-Tzu: 329
toy_terrier: 270
Test data category sizes:
Afghan_hound: 162
Blenheim_spaniel: 252
Chihuahua: 153
Japanese_spaniel: 214
Maltese_dog: 102
papillon: 184
Pekinese: 184
Rhodesian_ridgeback: 181
Shih-Tzu: 185
toy_terrier: 158


TRANFERS LEARNING


In [ ]:
# Data augmentation and preprocessing
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip('horizontal'),
    tf.keras.layers.RandomRotation(0.2),
    tf.keras.layers.RandomZoom(0.2),
    tf.keras.layers.RandomContrast(0.2),
    tf.keras.layers.Resizing(224, 224)
])

In [ ]:
preprocess_input = tf.keras.applications.resnet50.preprocess_input
IMG_SIZE = (224, 224)
IMG_SHAPE = IMG_SIZE + (3,)

base_model = tf.keras.applications.ResNet50(input_shape=IMG_SHAPE,
                                            include_top=False,
                                            weights='imagenet')

94765736/94765736 [==============================] - 1s 0us/step


In [ ]:
base_model.trainable = False

In [ ]:
# Build the model
inputs = tf.keras.Input(shape=(224, 224, 3))
x = data_augmentation(inputs)
x = preprocess_input(x)
x = base_model(x, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dropout(0.3)(x)
x = tf.keras.layers.Dense(128, activation='relu')(x)
x = tf.keras.layers.Dense(64, activation='relu')(x)
x = tf.keras.layers.Dense(32, activation='relu')(x)
outputs = tf.keras.layers.Dense(10, activation='softmax')(x)  # 10 classes


model = tf.keras.Model(inputs, outputs)

In [ ]:
base_learning_rate = 0.001
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])


In [ ]:
# Callbacks
early_stopping_cb = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
reduce_lr_cb = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=0.00001)


In [ ]:
# Fit the model
history = model.fit(x_train, y_train, epochs=15, validation_data=(x_test, y_test), callbacks=[early_stopping_cb, reduce_lr_cb])


Epoch 1/15
102/102 [==============================] - 1073s 11s/step - loss: 1.1008 - accuracy: 0.6373 - val_loss: 0.3594 - val_accuracy: 0.8980 - lr: 0.0010
Epoch 2/15
102/102 [==============================] - 1021s 10s/step - loss: 0.5303 - accuracy: 0.8207 - val_loss: 0.3074 - val_accuracy: 0.9003 - lr: 0.0010
Epoch 3/15
102/102 [==============================] - 1013s 10s/step - loss: 0.5204 - accuracy: 0.8237 - val_loss: 0.3004 - val_accuracy: 0.8975 - lr: 0.0010
Epoch 4/15
102/102 [==============================] - 1014s 10s/step - loss: 0.4339 - accuracy: 0.8487 - val_loss: 0.2560 - val_accuracy: 0.9206 - lr: 0.0010
Epoch 5/15
102/102 [==============================] - 955s 9s/step - loss: 0.4065 - accuracy: 0.8607 - val_loss: 0.2741 - val_accuracy: 0.9166 - lr: 0.0010
Epoch 6/15
102/102 [==============================] - 955s 9s/step - loss: 0.3420 - accuracy: 0.8850 - val_loss: 0.2760 - val_accuracy: 0.9161 - lr: 0.0010
Epoch 7/15
102/102 [==============================] - 10

In [ ]:
base_model.trainable = True

FINE TUNNING

In [ ]:
# Unfreeze the top layers of the model
fine_tune_at = 100  # Adjust based on ResNet50 structure
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate/10),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])

In [ ]:
fine_tune_epochs = 10
total_epochs = len(history.epoch) + fine_tune_epochs

history_fine = model.fit(x_train, y_train,
                         epochs=total_epochs,
                         initial_epoch=len(history.epoch),
                         validation_data=(x_test, y_test),
                         callbacks=[early_stopping_cb, reduce_lr_cb])

In [ ]:
# Plot the training and validation accuracy and loss side by side
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# Plot the training and validation accuracy
ax1.plot(history.history['accuracy'])
ax1.plot(history.history['val_accuracy'])
ax1.set_title('Model Accuracy')
ax1.set_ylabel('Accuracy')
ax1.set_xlabel('Epoch')
ax1.legend(['Train', 'Validation'], loc='upper left')

# Plot the training and validation loss
ax2.plot(history.history['loss'])
ax2.plot(history.history['val_loss'])
ax2.set_title('Model Loss')
ax2.set_ylabel('Loss')
ax2.set_xlabel('Epoch')
ax2.legend(['Train', 'Validation'], loc='upper left')

# Show the plots
plt.tight_layout()
plt.show()

In [ ]:
model.summary()

In [ ]:
# Save the model to a .h5 file
model.save('resnet50_model.h5')


In [ ]:
predictions = model.predict(x_test)
predicted_labels = np.argmax(predictions, axis=1)

# Find incorrectly predicted images
incorrect_indices = np.where(predicted_labels != y_test)[0]
plt.figure(figsize=(20, 20))
for i, incorrect_idx in enumerate(incorrect_indices[:30]):
    plt.subplot(10, 10, i + 1)
    plt.imshow(x_test[incorrect_idx])
    plt.title(f"True: {y_test[incorrect_idx]}, Pred: {predicted_labels[incorrect_idx]}")
    plt.axis('off')
plt.show()

In [ ]:
# prompt: plot dự đoán đúng sai

# Plot the first 25 correct predictions
plt.figure(figsize=(10, 10))
for i in range(25):
    plt.subplot(5, 5, i + 1)
    plt.imshow(x_test[i])
    plt.title(f"True: {y_test[i]}, Pred: {predicted_labels[i]}")
    plt.axis('off')
plt.show()

# Plot the first 25 incorrect predictions
plt.figure(figsize=(10, 10))
for i in range(25):
    incorrect_idx = incorrect_indices[i]
    plt.subplot(5, 5, i + 1)
    plt.imshow(x_test[incorrect_idx])
    plt.title(f"True: {y_test[incorrect_idx]}, Pred: {predicted_labels[incorrect_idx]}")
    plt.axis('off')
plt.show()


In [ ]:
# prompt: plot dự đoán đúng sai

# Plot the first 25 correct predictions
plt.figure(figsize=(10, 10))
for i in range(25):
    plt.subplot(5, 5, i + 1)
    plt.imshow(x_test[i])
    plt.title(f"True: {y_test[i]}, Pred: {predicted_labels[i]}")
    plt.axis('off')
plt.show()

# Plot the first 25 incorrect predictions
plt.figure(figsize=(10, 10))
for i in range(25):
    incorrect_idx = incorrect_indices[i]
    plt.subplot(5, 5, i + 1)
    plt.imshow(x_test[incorrect_idx])
    plt.title(f"True: {y_test[incorrect_idx]}, Pred: {predicted_labels[incorrect_idx]}")
    plt.axis('off')
plt.show()


TEST


In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

# Generate the confusion matrix
cm = confusion_matrix(y_test, predicted_labels)
# Normalize the confusion matrix
cm_norm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
# Set up the plot
fig, ax = plt.subplots(figsize=(8, 8))

# Create a heatmap and colorbar
cax = ax.matshow(cm, cmap=plt.cm.Blues)
fig.colorbar(cax)

# Set labels
ax.set(
    title="Confusion Matrix (Normalized to %)",
    xlabel="True label",
    ylabel="Predicted label",
    xticks=np.arange(len(categories)),
    yticks=np.arange(len(categories)),
    xticklabels=categories,
    yticklabels=categories,
)

# Label each cell with the corresponding value
for (i, j), z in np.ndenumerate(cm_norm):
    ax.text(j, i, f"{z * 100:.1f}%", ha="center", va="center", fontsize=10)

# Rotate x-axis labels to prevent overlapping
plt.xticks(rotation=45)

# Display the plot
plt.show()